# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [2]:
# Hacé la separación en esta celda
np.random.seed(123)
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [3]:
# Creá en esta celda la variable param_grid
param_grid = [
    { 'max_depth': [1,2,3,4,5], 
      'max_features': [1,2,3,4,5],
      'random_state':[0]}
]

**Importá** 'GridSearchCV' y 'DecisionTreeRegressor'.

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [5]:
# Importa y crea un GridSearchCV en esta celda
tree_regresor = DecisionTreeRegressor()
grid_search = GridSearchCV(tree_regresor,param_grid, cv=5, 
                           scoring='neg_mean_squared_error',
                          return_train_score= True)

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [6]:
# Hace el fit de grid search en esta celda
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid=[{'max_depth': [1, 2, 3, 4, 5],
                          'max_features': [1, 2, 3, 4, 5],
                          'random_state': [0]}],
             pre_dispatch='2*n_jobs', refit=True

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [7]:
grid_search.scorer_

make_scorer(mean_squared_error, greater_is_better=False)

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [8]:
results= pd.DataFrame(grid_search.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_random_state,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.004,0.001,0.001,0.000,1,1,0,"{'max_depth': 1, 'max_features': 1, 'random_st...",-997527974.201,-992475505.903,...,-979790172.137,25507079.717,24,-974522238.776,-975835495.262,-990365859.353,-972973324.041,-981374112.276,-979014205.942,6343554.490
1,0.003,0.000,0.001,0.000,1,2,0,"{'max_depth': 1, 'max_features': 2, 'random_st...",-997975870.246,-992475505.903,...,-979795655.597,25648380.710,25,-973762035.375,-975835495.262,-989776175.105,-972345322.505,-980838906.221,-978511586.894,6325344.028
2,0.003,0.000,0.001,0.000,1,3,0,"{'max_depth': 1, 'max_features': 3, 'random_st...",-986068693.940,-972814621.276,...,-967431345.965,26842619.528,17,-963242254.434,-962246910.451,-979824282.485,-960565397.833,-969695459.760,-967114860.993,7067932.886
3,0.002,0.000,0.001,0.000,1,4,0,"{'max_depth': 1, 'max_features': 4, 'random_st...",-986068693.940,-972814621.276,...,-967431345.965,26842619.528,17,-963242254.434,-962246910.451,-979824282.485,-960565397.833,-969695459.760,-967114860.993,7067932.886
4,0.002,0.000,0.001,0.000,1,5,0,"{'max_depth': 1, 'max_features': 5, 'random_st...",-986068693.940,-972814621.276,...,-967431345.965,26842619.528,17,-963242254.434,-962246910.451,-979824282.485,-960565397.833,-969695459.760,-967114860.993,7067932.886
5,0.002,0.000,0.001,0.000,2,1,0,"{'max_depth': 2, 'max_features': 1, 'random_st...",-996490372.809,-992306520.398,...,-978942683.108,25972632.706,23,-973137294.201,-974299503.542,-989317535.990,-971604933.549,-979850510.815,-977641955.619,6466938.826
6,0.002,0.000,0.001,0.000,2,2,0,"{'max_depth': 2, 'max_features': 2, 'random_st...",-996946973.633,-976087254.774,...,-976100751.575,23877009.024,21,-972652904.691,-968624656.598,-988138914.877,-971407817.856,-979671314.336,-976099121.672,7035190.857
7,0.002,0.000,0.001,0.000,2,3,0,"{'max_depth': 2, 'max_features': 3, 'random_st...",-969811583.866,-755794407.028,...,-908413692.285,80622443.030,9,-940110807.542,-730356341.399,-955363881.999,-937848000.307,-950240102.523,-902783826.754,86453221.787
8,0.002,0.000,0.001,0.000,2,4,0,"{'max_depth': 2, 'max_features': 4, 'random_st...",-982713573.930,-974503916.115,...,-966098817.461,26853962.890,16,-959163708.924,-959198007.671,-975114537.298,-955685688.730,-965808588.254,-962994106.175,6889432.959
9,0.009,0.012,0.001,0.000,2,5,0,"{'max_depth': 2, 'max_features': 5, 'random_st...",-986429011.604,-971604076.963,...,-965565624.511,29901901.022,15,-959271718.195,-958800549.958,-977522392.608,-956115730.767,-966909058.601,-963723890.026,7778035.370


De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3. 

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [9]:
# Mostrás los resultados en esta celda
print("Mejor conjunto de parámetros:", grid_search.best_params_, end="\n\n")
print("Mejor score:", grid_search.best_score_, end="\n\n")


Mejor conjunto de parámetros: {'max_depth': 3, 'max_features': 3, 'random_state': 0}

Mejor score: -729969370.3438246



Convertimos a RMSE.

In [10]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

In [11]:
print("Mejor score:", nmsq2rmse(grid_search.best_score_), end="\n\n")

Mejor score: 27017.95



__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

In [12]:
# Creá en esta celda la variable param_grid
param_grid = [
    { "min_samples_split": [2, 10, 20], 
      "max_depth": [None, 2, 5, 10, 15],
      "min_samples_leaf": [1, 5, 10, 15],
      "max_leaf_nodes": [None, 5, 10, 20]}
]

In [13]:
tree_regresor = DecisionTreeRegressor()
grid_search = GridSearchCV(tree_regresor,param_grid, cv=5, 
                           scoring='neg_mean_squared_error',
                          return_train_score= True)

In [ ]:
# Hace el fit de grid search en esta celda
grid_search.fit(X_train, y_train)

In [ ]:
# Mostrás los resultados en esta celda
print("Mejor conjunto de parámetros:", grid_search.best_params_, end="\n\n")
print("Mejor score:", grid_search.best_score_, end="\n\n")

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [ ]:
optimised_decision_tree = grid_search.best_estimator_
optimised_decision_tree

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [ ]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [ ]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [ ]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [ ]:
predicciones.head(10)